In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
from src.train_mlm import get_data

In [2]:
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

with initialize(config_path="../conf"):
    cfg = compose(config_name="config.yaml")
    print(cfg)

dataset = get_data(cfg)

{'architecture': 'mlm', 'mlruns_dir': 'mlruns', 'overwrite_data_cache': True, 'data_path': 'data', 'data_subfolder': 'data/all_data_en', 'download_url': 'https://crisisnlp.qcri.org/data/crisis_datasets_benchmarks/crisis_datasets_benchmarks_v1.0.tar.gz', 'task': 'informativeness', 'language': 'EN', 'seed': 42, 'gpu': {'fp16': True, 'fp16_opt_level': 'O2', 'half_precision_backend': 'auto'}, 'debugging_mode': False, 'mode': {'name': 'train', 'do_train': True, 'do_eval': True, 'continue_training': False, 'per_device_train_batch_size': 32, 'per_device_eval_batch_size': 32, 'learning_rate': 3e-05, 'weight_decay': 0.1, 'num_train_epochs': 60, 'pad_to_max_length': False, 'max_seq_length': 'None', 'mlm_probability': 0.15, 'evaluation_strategy': 'steps', 'eval_steps': 500, 'logging_steps': 5000, 'early_stopping': True, 'patience': 50, 'save_model': True}, 'model': {'name': 'distilbert', 'pretrained_model': 'distilbert-base-uncased'}}


Reusing dataset crisis_bench_dataset (/export/home/kraft/.cache/huggingface/datasets/crisis_bench_dataset/informativeness/1.0.0/2e8a367b9209a9e88eb3aeca65dc626e3d123def2e180cfec317671f187fa562)


  0%|          | 0/3 [00:00<?, ?it/s]

Loaded dataset with DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 109796
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 16008
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 31095
    })
})


In [13]:
print(next(iter(dataset['test']['text'])))

Tesla finishes first of many solar projects in Puerto Rico Read more : http http


In [28]:
test_set_path = "/export/home/kraft/infhome/CrisisLM/data/data/all_data_en/crisis_consolidated_informativeness_filtered_lang_en_test.tsv"
delim = "\t"

In [29]:
from src.TweetNormalizer import normalizeTweet

with open(test_set_path, "r", newline=None, encoding='utf-8', errors='replace') as f:
    for i, line in enumerate(f):
        line = line.strip()
        if line == "":
            continue
        row = line.split(delim)
        text = row[3].strip()
        text = normalizeTweet(text)
        label = row[6]
        if label == 0:
            continue
        print(text)
    


text
Tesla finishes first of many solar projects in Puerto Rico Read more : http http
@user I 'm like 2 miles away . We always get the day off for the marathon . I can see the area from my window .
RT @user : BOSTON MARATHON EXPLOSIONS : 1st explosion reported on North side of Boylston Street by the the finish line . 2nd explosion sec ...
Make that 5 co-workers thanks to Bailee .
BRING ALL YOUR WORRIES TO HIM BECAUSE HE CARES FOR YOU #AyokoNaSana #ThingsToDoPagMalamig #ForevermoreTheGrandeBash #RubyPH
" #Intern #Accounting Internship ( Finance and Economics ) - San Antonio , TX Job : Scottrade - S ... http #Texas #Internship "
My wish for you is that this life becomes all that you want it to
Loan Upheaval Is The Way In Which Oneself Can Save Your House Leaving out Being Foreclosed On ... TEJc
Spring in Paris be like @user Montmartre http
. @user : Over 15 million people exposed to strong shaking after #Mexicoearthquake . Report : http http
Update : Vanuatu Hit Hard by Cyclone http
RT @

@user : Thirsty ass it just be like that sometimes .
#India news - Odisha Flood Toll Mounts to 45 , Water Level Goes Down http
I would like have some information about this time please .
My boo Mena i miss her @user New York State DMV http
RT @user : We are under a tornadoe watch . Stay safe friends in North Texas . / / Tornado Warning now ! You and the fam be safe .
Happy 1st Large Milk Chocolate Letters-Anniversary Birthday Gift for Boyfriend Girlfriend-Made to Order http
Marathon bombing suspect dead , another on run : WATERTOWN , Mass . ( AP ) ? Two suspects in the Boston Marathon ... http
RT @user : LIVE VIDEO : NYC Mayor Michael Bloomberg updates residents on Hurricane #Sandy http
@user Deadly tornado rips through Oklahoma City : US president declares major disaster with 91 people feared dead and scores mo ...
Almost every day , the clinic dispenses monthly rations of Plumpy'Sup , a peanut butter-based ready-to-eat paste packed with vitamins , minerals and other micronutrients sp

Damn tornado warning do n't mean shit 2 me . Motherfucka , I have survived Katrina 's bitch ass , 2 tornados & a blizzard . Just another day 2 me
Welcome home from Ireland Strings Attached ! http
Something to soothe the sore throat with dinner . #teroldego #aubonclimat #missingcaliwinetrips http
#Cherokee #indian host Chief many feathers ! @user Gary & . Miguels Penthouse apt http
#Park near my house o - o #bigwet #flood #queensland #brisbane http
I honestly think Cordell is gay
To ensure safety , local railway authorities closed the tracks on one side of the rail line .
@user : Im really bored in bedddddd . Ahhhhhh I keep changing positions every 5 mins lmao
I never thought I 'd be back in the Laura Steve part of my life .
" Sayville http beat Westhampton Beach http , 35-7 on Sat Oct 29 # #football "
K well this is sweet
Another very dangerous storm possible #tornado just NE of Leon #Oklahoma . Seek shelter NE of this cell ! #OKWX Follow 4 updates .
@user In Texas , nahdha would be co

From the train , showing the smoke filled sky from the #Lithgow #nswfires http
#earthquake : M 2.3 , Southern California http
RT @user : #RubyPH cuts power lines in Southern Luzon , Visayas http http
Houston reporter Brandi Smith saved a life while covering Hurricane Harvey . http http
The foods that they distribute in the country.We call S . O . S .
@user @user You 're a bit slow . I counselled 4th Sep : " Like #Harvey , incredible amount of standing http
Breaking news . .. police have a suspect in Boston bombing
Food is getting very costly in PAP following the earthquake . Rice is now 250 gourdes a measuree it was 200 gourdes before . Corn is 125 gourdes , compared to 100 gourdes
@user how OLD are you ? Making fun of an icon - not cool .
@user @user a flood of fictions , hope its not done with tax-payers amounts somehow
Day by day #HurricaneMaria #stateofmind #workingonit #noteasy #thursday #alive #blessed @user http
Sitting here doing nothing for a few hours , decided to do homework

Mayor says 14 dead in Texas fertilizer plant blast : WEST , Texas - The death toll in the West , Texas explosion ... http
Each pack consists of a kitchen set , bed sheet , men 's and women 's clothing , towel , plastic mat , kerosene stove , candles and 10 kilograms of rice .
@user @user @user You have confused #MRSA with #MERS . One is #antibiotic-resist staph , big US prblm . Other = virus
@user : Mr . President , did you spread Ebola to the US because you 're an exec order-issuing tyrant ? #WHPressCorpsQuestions lmao
UPDATE : 26y / o QLD yoga teacher Nadwa Natalie Alasfar is still missing after the #NepalQuake . #9News http
Iran in Mourning After Earthquake that Killed 430 http http
Food Emergency Light Water You can never be too ready . #RubyPH
RT @user : PAGASA : #RubyPH / #Hagupit has a maximum sustained winds of 120 kph near the center & gustiness of up to 150 kph .
@user : Evac 's after rain triggers a landslide in Syd , Clive Palmer slammed over comments about China & firefigh

...

National Guard Helps Out Residents Of U . S . Virgin Islands After Hurricane Maria - http http
Colorado Springs Fire Consuming Homes Forces 34500 to Flee - San Francisco Chronicle : NDTVColorado Springs Fire ... http
Here it comes . Dickhead Mundine says he was robbed . Ca n't believe how much of a cock this bloke is . Tools of the year Mundine and SBW .
Will be they back to manufacture the CIN ( Identification National Card )
Errrr . Connection please ! The effect of #RubyPH
Smoothie de jengibre con mora http #Gastronoma http
" After all these allegations one sees why #Figo pulled out . #FIFA ##Bribes #FBI "
As Jacen approached the Sow Stateroom , one of the guards began to moan for help . The fellow 's torso had been cleaved at
@user there 's nothing you could have done .
#src @user : #AmericanRedCross : #loc Greater Ozarks Chapter #needs cleaning supplies . #info PLEASE NO MORE BOTTLED WATER . #Joplin
Scary stuff in San Antonio . Stars head coach Dan Hughes was run into on the sidel

I need to know what the new plan united nations has for haiti ?
Forest City native shares experience evacuating from California wildfires http http
RT @user : #MERS is a disease MERS-CoV is a virus A virus is not a disease A disease is not a virus A virus can cause a disease #wordsma
I 'm just like ... i CANT DEAL
@user @user No , best keep it quiet so wealthy people do n't muscle in on quality near free govt schools and ruin it for us .
RT @user : Saving our doggie pals ! #harvey #doglovers #doggiepals http
Conning Builds Strong Case for Portfolio #Diversification for Property-Casualty Insurers http
If you are a heardache.but , you take a drugs.he still heardache.you have one months half.what do make .
So proud of our kindergarten Eagles taking pride in helping keep our campus beautiful following Irma ! @user http
@user At first I thought you were referencing #abflood : Drop everything now / Meet me in the pouring rain . But then ... nope .
Though the rains eased Monday , many localit

@user dude I have no way There . Their closing the island soon . No ones gonna be able to grt on
Spain Train Derailment : Spain Train Derailment http #Atlanta #GA
RT @user : Drinking water needed in bemina @user @user #KashmirFlood http
Germany bans Russian meteor videos on YouTube ; thinktanks of climate denial ; and abd supersonic pingpong : http ...
Emergency Hotlines ! #floodPH #rescuePH #Hagupit #RubyPH #safePH Please be safe everyone . http
i hear , that you and your band , are throwing out your guitars , and buying The Russian Meteor
HAGUPIT SLAMS PHILIPPINES ... - HAGUPIT SLAMS PHILIPPINES ... ( First column , 6th story , link ) http
Not me im enjoying relaxing ? ? RT @user : All y'all was wishin for hurricane sandy and now y'all sick
id like to know if we can have two natural phenomenon at the same time ?
Tropical cyclone batters Vanuatu ; dozens feared dead http
Michigan State volleyball to only play one match in Florida due to Hurricane Irma http http
Rangers game in a torna

Marathoners been rerouted a bloxk before Race finish - city in mayhem - race still going . Our runner Em ok . Still have n't seen her yet
#Brisbane #river #boats #Australia @user Brisbane River Walk http
Keep on praying for the safety of every Filipinos . Our God is bigger than #RubyPH
Peat consists of dense organic materials accumulated in lowland areas for 7,000 years or more and may be as much as 7 meters deep .
Scorpion #M6 #replay #jadore _C__C
RT @user : @user Can you please RT ? #clutha #glasgow #charity Charity pub football tournament in & around Glasgow ! http
RT @user : RT @user : #BREAKING : Source says large swath of West , Texas leveled in fertilizer plant explosion .
RT @user : Be inspired by @user awesome blog post : http What are your epic wins ? #caedchat
Send me all the pertinent information , please .
First Time Playing Hearthstone on PC Thoughts http
#UGC RT @user : The rain is getting stronger here in #Malapascua #Cebu and no more electricity . #PabloPH http
RT @us

The Five Fatal Flaws in the Iran Deal http via @user
if u woke up from a coma today & turned on the news u would literally have no idea 5000 people just died in #Nepal CNN / FOX & MSNBC R a joke
@user - u were a guest star in my dream last night , & 2day I noticed Marcus Garvey Park on a NYC #hurricane map ! http
PG & E Hopes To Restore Power In Wildfire Areas By Mond ... http via @user http
RT @user : Due to the CP Rail incident and road closures , we ask you to stay off city thoroughfares unless necessary . #yycflood
A tornado spawned by #HurricaneIrma Piece turned a piece of roof into a spear in Palm Bay #FOX35 http
Norte Dame spring game on TV , gets your mind right #fightingirish
Alberta must do a ' much better job of forecasting ' after failing to sound flood ... - Calgary Herald : Calgary ... http
I 'm too legit to quit
YaAllah another one ! We already have #Pmln #MQM #ANP :/ y these #earthquake now :(
@user the mass murder got her hot and bothered but at heart she was always a 

In [ ]:
# Masking https://github.com/huggingface/transformers/blob/v4.16.2/src/transformers/data/data_collator.py#L846
# iterate through words and add all except  spec. chars and punctuation to candidate list
# random.shuffle(cand_indexes) -> seed!
# num_to_predict = min(max_predictions, max(1, int(round(len(input_tokens) * self.mlm_probability))))
masked_lms = []
        covered_indexes = set()
        for index_set in cand_indexes:
            if len(masked_lms) >= num_to_predict:
                break
            # If adding a whole-word mask would exceed the maximum number of
            # predictions, then just skip this candidate.
            if len(masked_lms) + len(index_set) > num_to_predict:
                continue
            is_any_index_covered = False
            for index in index_set:
                if index in covered_indexes:
                    is_any_index_covered = True
                    break
            if is_any_index_covered:
                continue
            for index in index_set:
                covered_indexes.add(index)
                masked_lms.append(index)

        if len(covered_indexes) != len(masked_lms):
            raise ValueError("Length of covered_indexes is not equal to length of masked_lms.")
        mask_labels = [1 if i in covered_indexes else 0 for i in range(len(input_tokens))]
        return mask_labels
# end product should be a .tsv file with inputs and labels --> random. seed to make test set replicable